1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 너무 오래 걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달달

In [1]:
%pip install --upgrade --quiet  docx2txt langchain-community

Note: you may need to restart the kernel to use updated packages.


In [41]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 토큰 수
    chunk_overlap=100,  # 겹치는 토큰 수
)

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)
print(f"문서의 길이: {len(document_list)}")

문서의 길이: 276


In [42]:
# embedding
from langchain_upstage import UpstageEmbeddings
from dotenv import load_dotenv
load_dotenv()

embedding = UpstageEmbeddings(model="solar-embedding-1-large")


In [19]:
print(embedding)

client=<openai.resources.embeddings.Embeddings object at 0x00000251827410A0> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000025186656300> model='solar-embedding-1-large' dimensions=None upstage_api_key=SecretStr('**********') upstage_api_base='https://api.upstage.ai/v1/solar' embedding_ctx_length=4096 embed_batch_size=10 allowed_special=set() disallowed_special='all' chunk_size=1000 max_retries=2 request_timeout=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None http_client=None http_async_client=None


In [54]:
len(document_list)

276

In [55]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

# Initialize Pinecone
pinecone_api_key = os.getenv("PINECONE_API_KEY")
index_name = 'tax-index'

# Split documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunked_documents = text_splitter.split_documents(document_list)
print(f"Chunked documents length: {len(chunked_documents)}")

# Initialize the PineconeVectorStore
database = PineconeVectorStore.from_documents(
    documents=[],  # Start with an empty list
    embedding=embedding,
    index_name=index_name
)

# Upload documents in batches
batch_size = 100
for i in range(0, len(chunked_documents), batch_size):
    print(f'index: {i}, batch size: {batch_size}')
    batch = chunked_documents[i:i + batch_size]
    database.add_documents(batch)  # Add documents to the existing database

index: 0, batch size: 100
index: 100, batch size: 100
index: 200, batch size: 100


In [56]:
len(chunked_documents)

276

In [45]:
query = "연봉 9천만원 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3)

In [46]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [47]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

d:\inflearn-rag-notebook-main\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [48]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [49]:
ai_message = qa_chain.invoke({"query": query})

In [50]:
ai_message

{'query': '연봉 9천만원 직장인의 소득세는 얼마인가요?',
 'result': '연봉 9천만원의 급여액을 가진 직장인의 경우, 소득세는 총급여액 7천만원 초과 1억2천만원 이하인 경우의 공제세액 계산법을 따릅니다. 이 경우 공제세액은 66만원에서 (총급여액 - 7천만원)의 1/2을 뺀 금액으로 계산되며, 이 금액이 50만원보다 적을 경우 50만원으로 합니다. 따라서 해당 직장인의 소득세는 공제세액을 제외한 나머지 금액에 대해 계산됩니다. 정확한 소득세액은 추가적인 공제 및 세액 정보를 통해 계산 가능합니다.'}